# GPX Reader - GPS Exchange Format

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys
import time

import gpxpy
import numpy as np

import unittest

from file_reader import FileReader

## Main Class

In [2]:
class GpxReader(FileReader):
    '''GPX file - GPS Exchange Format'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)

        self.points = []


    def load(self):
        '''Load file into memory'''

        with open(self.filename, "rb") as f:
            gpx = gpxpy.parse(f)
            
            counts = \
            {
                'latitude': 0,
                'longitude': 0,
                'elevation': 0,
                'time': 0,
                'speed': 0,
                'course': 0,
                'satellites': 0,
                'horizontal_dilution': 0
            }
    
            for track in gpx.tracks:
                for segment in track.segments:
                    for point in segment.points:
                        self.points.append(point)
                        for item in counts:
                            if getattr(point, item) is not None:
                                counts[item] += 1

            self.numRecords = len(self.points)
            if self.numRecords > 0:
                self.consumePoints(counts)


    def consumePoints(self, counts):
        '''Consume points - convert to standard data structure'''

        # Copy latitude
        iterable = (getattr(point, 'latitude') for point in self.points)
        self.data['lat'] = np.fromiter(iterable, np.double)
            
        # Copy longitude
        iterable = (getattr(point, 'longitude') for point in self.points)
        self.data['lon'] = np.fromiter(iterable, np.double)
            
        # Copy elevation, if present
        if counts['elevation'] > 0:
            iterable = (getattr(point, 'elevation') for point in self.points)
            self.data['ele'] = np.fromiter(iterable, np.single)

        # Copy timestamp, if present
        if counts['time'] > 0:
            iterable = (getattr(point, 'time').timestamp() for point in self.points)
            self.data['timestamp'] = np.fromiter(iterable, np.double)
        
        # Copy SOG, if present
        if counts['speed'] > 0:
            iterable = (getattr(point, 'speed') for point in self.points)
            self.data['sog'] = np.fromiter(iterable, np.double)

        # Copy COG, if present - COROS uses <cog>...</cog> which is incorrect and is not supported by gpxpy
        if counts['course'] > 0:
            iterable = (0 if getattr(point, 'course') is None else getattr(point, 'course') for point in self.points)
            self.data['cog'] = np.fromiter(iterable, np.double)

        # Copy satellites, if present with logic to handle <sat></sat>
        if counts['satellites'] > 0:
            iterable = (0 if getattr(point, 'satellites') is None else getattr(point, 'satellites') for point in self.points)
            self.data['sat'] = np.fromiter(iterable, np.uint8)

        # Copy HDOP, if present
        if counts['horizontal_dilution'] > 0:
            iterable = (getattr(point, 'horizontal_dilution') for point in self.points)
            self.data['hdop'] = np.fromiter(iterable, np.double)

## Unit Tests

In [3]:
class TestCorosData(unittest.TestCase):
    '''Class to test COROS data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(corosReader.numRecords, 3809)

        for fieldName in corosReader.data:
            self.assertEqual(corosReader.data[fieldName].size, 3809)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(corosReader.data['hdop'].min(), 0.7)
        self.assertEqual(corosReader.data['hdop'].max(), 1.6)


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(corosReader.data['sat'].min(), 0)
        self.assertEqual(corosReader.data['sat'].max(), 14)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(corosReader.data['timestamp'].min(), 1649672017.0)
        self.assertEqual(corosReader.data['timestamp'].max(), 1649677777.0)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(corosReader.data['lat'].min(), 50.5705683)
        self.assertEqual(corosReader.data['lat'].max(), 50.5832829)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(corosReader.data['lon'].min(), -2.4620381)
        self.assertEqual(corosReader.data['lon'].max(), -2.4559603)


    def testCog(self):
        '''Test the course over ground is as expected'''

        if 'cog' in corosReader.data:
            self.assertEqual(corosReader.data['cog'].min(), 0)
            self.assertEqual(corosReader.data['cog'].max(), 359)

In [4]:
class TestSbnData(unittest.TestCase):
    '''Class to test GT-31 SBN data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(sbnReader.numRecords, 4161)

        for fieldName in sbnReader.data:
            self.assertEqual(sbnReader.data[fieldName].size, 4161)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(sbnReader.data['hdop'].min(), 0.8)
        self.assertEqual(sbnReader.data['hdop'].max(), 2.0)


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(sbnReader.data['sat'].min(), 4)
        self.assertEqual(sbnReader.data['sat'].max(), 10)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(sbnReader.data['timestamp'].min(), 1649672182)
        self.assertEqual(sbnReader.data['timestamp'].max(), 1649678792)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(sbnReader.data['lat'].min(), 50.571016)
        self.assertEqual(sbnReader.data['lat'].max(), 50.5833319)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(sbnReader.data['lon'].min(), -2.4620455)
        self.assertEqual(sbnReader.data['lon'].max(), -2.4563038)


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(np.round(sbnReader.data['ele'].min() + 3.02, 2), 0)
        self.assertEqual(np.round(sbnReader.data['ele'].max() - 11.93, 2), 0)


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(sbnReader.data['sog'].min(), 0.13)
        self.assertEqual(sbnReader.data['sog'].max(), 16.83)


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(sbnReader.data['cog'].min(), 0.01)
        self.assertEqual(sbnReader.data['cog'].max().round(2), 359.92)

In [5]:
class TestSbpData(unittest.TestCase):
    '''Class to test GT-31 SBP data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(sbpReader.numRecords, 4162)

        for fieldName in sbpReader.data:
            self.assertEqual(sbpReader.data[fieldName].size, 4162)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(sbpReader.data['hdop'].min(), 0.8)
        self.assertEqual(sbpReader.data['hdop'].max(), 2.0)


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(sbpReader.data['sat'].min(), 0)
        self.assertEqual(sbpReader.data['sat'].max(), 10)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(sbpReader.data['timestamp'].min(), 1649672161)
        self.assertEqual(sbpReader.data['timestamp'].max(), 1649678793)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(sbpReader.data['lat'].min(), 50.5710156)
        self.assertEqual(sbpReader.data['lat'].max(), 50.583341)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(sbpReader.data['lon'].min(), -2.4620455)
        self.assertEqual(sbpReader.data['lon'].max(), -2.4563038)


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(np.round(sbpReader.data['ele'].min() + 3.06, 2), 0)
        self.assertEqual(np.round(sbpReader.data['ele'].max() - 11.93, 2), 0)


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(sbpReader.data['sog'].min(), 0.01)
        self.assertEqual(sbpReader.data['sog'].max(), 16.83)


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(sbpReader.data['cog'].min(), 0.01)
        self.assertEqual(sbpReader.data['cog'].max().round(2), 359.92)

In [6]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220411', 'APEX_Pro_Speedsurfing20220411111317.gpx')
    corosReader = GpxReader(filename)

    filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220411_111600.gpx')
    sbnReader = GpxReader(filename)

    filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220512_094254_DLG.gpx')
    sbpReader = GpxReader(filename)

    pc1 = time.perf_counter()
    corosReader.load()
    sbnReader.load()
    sbpReader.load()
    pc2 = time.perf_counter()

    print("\nGPX files loaded in in %0.2f seconds" % (pc2 - pc1))


GPX files loaded in in 0.76 seconds


In [7]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

.........................
----------------------------------------------------------------------
Ran 25 tests in 0.014s

OK
